# Introduction
State notebook purpose here

### Get source folder and append to sys directory

In [46]:
from __future__ import print_function
import os
import sys
PROJ_ROOT = os.path.join(os.pardir)
print(os.path.abspath(PROJ_ROOT))
src_dir = os.path.join(PROJ_ROOT, "src")
sys.path.append(src_dir)
# Data path example
#pump_data_path = os.path.join(PROJ_ROOT,
#                              "data",
#                              "raw",
#                              "pumps_train_values.csv")

/mnt/4ba37af6-51fd-47bc-8321-8c500c229114/study/School/KHOA LUAN TOT NGHIEP/runnable_program


In [41]:
sys.path

['/mnt/4ba37af6-51fd-47bc-8321-8c500c229114/study/School/KHOA LUAN TOT NGHIEP/runnable_program/notebooks',
 '/home/nam/.vscode/extensions/ms-toolsai.jupyter-2021.5.745244803/pythonFiles',
 '/home/nam/Development/anaconda3/lib/python3.8/site-packages/_pdbpp_path_hack',
 '/home/nam/program/spark-3.0.0-preview-bin-hadoop2.7/python',
 '/home/nam/.vscode/extensions/ms-toolsai.jupyter-2021.5.745244803/pythonFiles',
 '/home/nam/.vscode/extensions/ms-toolsai.jupyter-2021.5.745244803/pythonFiles/lib/python',
 '/home/nam/Development/anaconda3/lib/python38.zip',
 '/home/nam/Development/anaconda3/lib/python3.8',
 '/home/nam/Development/anaconda3/lib/python3.8/lib-dynload',
 '',
 '/home/nam/.local/lib/python3.8/site-packages',
 '/home/nam/Development/anaconda3/lib/python3.8/site-packages',
 '/home/nam/Development/anaconda3/lib/python3.8/site-packages/IPython/extensions',
 '/home/nam/.ipython',
 '../src',
 '../../src']

### Imports
Import libraries and write settings here.

In [43]:
# Data manipulation
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn
from tensorflow import keras
from tensorflow.keras import layers
import glob
import xarray as xr
from sklearn.preprocessing import MinMaxScaler
import holidays

# Extract Data
%aimport features.extract_features
from features import extract_features
%aimport data.create_load_transform_processed_data
from data import create_load_transform_processed_data
# Make dataset

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30
# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from IPython import get_ipython
ipython = get_ipython()
# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload
%autoreload 1
# Use %aimport module to reload each module

# Visualizations
import matplotlib.pyplot as plt

# Analysis/Modeling
Do work here

### Get interim data

In [47]:
_interim_data_path = os.path.join(PROJ_ROOT,
                                  "data",
                                  "interim")
_interim_files = glob.glob(_interim_data_path + '/*.csv')

interim_df = pd.DataFrame()
for file in _interim_files:
    print('Currently processing file \n{}'.format(file))
    interim_df = interim_df.append(pd.read_csv(file, parse_dates=True, index_col=['site_id', 'time'],
                                                     dtype={'CO': np.float64, 'NO2': np.float64, 'PM25': np.float64,
                                                            'AQI_h': np.float64, 'AQI_h_I': np.int, 'site_id': np.int}))
# Site 16 have many inconsistency in data so we remove it
interim_df = interim_df[(interim_df.index.get_level_values(0) != 16)]
# Get only columns we need
interim_df = interim_df[['PM25', 'AQI_h', 'AQI_h_Polutant', 'AQI_h_I',
       'AQI_h_label', 'Continous length']]
# Ho Chi Minh data is on site 49
hanoi_df = interim_df[(interim_df.index.get_level_values(0) != 49)].copy()
hcm_df = interim_df[(interim_df.index.get_level_values(0) == 49)].copy()

Currently processing file 
../data/interim/30.csv
Currently processing file 
../data/interim/9.csv
Currently processing file 
../data/interim/32.csv
Currently processing file 
../data/interim/11.csv
Currently processing file 
../data/interim/40.csv
Currently processing file 
../data/interim/28.csv
Currently processing file 
../data/interim/49.csv
Currently processing file 
../data/interim/46.csv
Currently processing file 
../data/interim/10.csv
Currently processing file 
../data/interim/8.csv
Currently processing file 
../data/interim/25.csv
Currently processing file 
../data/interim/7.csv
Currently processing file 
../data/interim/16.csv
Currently processing file 
../data/interim/42.csv
Currently processing file 
../data/interim/44.csv
Currently processing file 
../data/interim/37.csv
Currently processing file 
../data/interim/1.csv
Currently processing file 
../data/interim/13.csv
Currently processing file 
../data/interim/31.csv
Currently processing file 
../data/interim/26.csv
Curr

### Create input for model from interim data and put in to processed

In [48]:
_processed_data_path = os.path.join(PROJ_ROOT,
                                   "data",
                                   "processed")

def create_input_for_model(df, timesteps=[1], target_hour=[1], output_path=None):
    """From interim dataframe:
        + add features
        + split into chunks according to timesteps
        + compressed and saved to output_path
        + estimate number of created dataset = timesteps * target_hour
    Parameters
    ----------
    df : pandas.DataFrame
        Contains interim data.
    timesteps : list of integer
        Each timestep represent 1 dataset
    target_hour : list of integer
        the label for each timesteps
    output_path : string
        Destination directory the dataset will be created
    """
    if output_path == None:
        output_path == os.path.join(PROJ_ROOT,
                                   "data",
                                   "processed")
    for timesteps in timesteps:
        for target_hour in target_hour:
            # Create train, dev, test data
            final_df = extract_features.add_features(df).copy()
            train_df, test_df = extract_features.generate_train_test_set_by_time(final_df)
            train_df, dev_df = extract_features.generate_train_test_set_by_time(train_df)
            train, y_train, multiclass_y = extract_features.data_preprocessing(train_df, target_hour, timesteps=timesteps)
            test, y_test, multiclass_y_test = extract_features.data_preprocessing(test_df, target_hour, timesteps=timesteps)
            dev, y_dev, multiclass_y_dev = extract_features.data_preprocessing(dev_df, target_hour, timesteps=timesteps)

            # Save data to file
            create_model_input.reshape_array_and_save_to_path(train, y_train, path=output_path, timesteps=timesteps, target_hour=target_hour, data_type="train")
            create_model_input.reshape_array_and_save_to_path(dev, y_dev, path=output_path, timesteps=timesteps, target_hour=target_hour, data_type="dev")
            create_model_input.reshape_array_and_save_to_path(test, y_test, path=output_path, timesteps=timesteps, target_hour=target_hour, data_type="test")
    print("Input have been created")
create_input_for_model(hanoi_df, output_path=_processed_data_path+"/hanoi")
create_input_for_model(hcm_df, output_path=_processed_data_path+"/hcm")

Feature shape:  (127813, 1, 68)
Label shape:  (127813,)
Feature shape:  (16363, 1, 68)
Label shape:  (16363,)
Feature shape:  (14744, 1, 68)
Label shape:  (14744,)
Data array:
shape of arr:  (127813, 1, 68)
shape of loaded_array:  (127813, 1, 68)
Yes, both the arrays are same
Label array:
shape of arr:  (127813,)
shape of loaded_array:  (127813,)
Yes, both the arrays are same
Data array:
shape of arr:  (14744, 1, 68)
shape of loaded_array:  (14744, 1, 68)
Yes, both the arrays are same
Label array:
shape of arr:  (14744,)
shape of loaded_array:  (14744,)
Yes, both the arrays are same
Data array:
shape of arr:  (16363, 1, 68)
shape of loaded_array:  (16363, 1, 68)
Yes, both the arrays are same
Label array:
shape of arr:  (16363,)
shape of loaded_array:  (16363,)
Yes, both the arrays are same
Input have been created


# Results
Show graphs and stats here

# Conclusions and Next Steps
Summarize findings here